# Learning the topology of a Bayesian Network from a database of cases using the K2 algorithm

In [ ]:
library(tidyverse)

# Installed just by typing install.packages('bnlearn')
library(bnlearn)

# This was a bit more complicated to install:
# install.packages("BiocManager")
# BiocManager::install("Rgraphviz")
library(Rgraphviz)

## Some theory

The problem of finding the best belief-network structure $B_S$ is a maximization problem of the quantity:

$$P(B_S|D)\propto P(B_S,D)$$

$$\max{\left[P(B_S|D)\right]}\leftrightarrow \max{\left[P(B_S,D)\right]}$$

Robinson formula computes the number of possible Networks given the number of nodes. It is relevant to show how quickly the problem tackled scales when the number of nodes _n_ increases.

As shown in the next cell, the complexity of this task is exponential.

In [ ]:
# Robinson recursive formula to compute the number of possible belief-network stuctures
# that contains n nodes
# From: A Bayesian Method for the Induction of Probabilistic Networks from Data, p. 319
n.networks.structure <- function(n)
    {
        if(n <= 1){return(1)}
        else
            {
                i <- 1
                res <- 0
                while(i <= n)
                    {
                        res <- res + ((i%%2)*2 - 1)*(choose(n,i)*(2^(i*(n-i)))*n.networks.structure(n-i))
                        i <- i + 1
                    }

                return(res)
            }
    }

for(i in 1:8)
    {
        cat('For n=',i,'the number of possible structures is: ',n.networks.structure(i),'\n')
    }


$$P(B_S,D)=P(B_S)\prod_{i=1}^n\prod_{j=1}^{q_i}\frac{(r_i-1)!}{(N_{ij}+r_i - 1)!}\prod_{k=1}^{r_i}N_{ijk}!$$

* $P(B_S)$ is the prior probability of the model, if we assume every model to be as likely as the others then it is just a constant
* $r_i$ is the number of possible value assignments of the i-th variable
* $\pi_i$ is the set of parents for the node i
* $q_i$ the number of unique instantiations of $\pi_i$ relative to D
* $w_{ij}$ denotes the jth unique instantiation of $\pi_i$ relative to D
* $N_{ijk}$ is the number of cases in D in which the variable $x_i$ has the value $v_{ik}$ and $\pi_i$ is instantiated as $w_{ij}$
* $N_{ij}=\sum_{k=1}^{r_i}N_{ijk}$

In [ ]:
ex <- read.table("./dataset/cooper.txt", header = TRUE, stringsAsFactors = TRUE)
ex

In [ ]:
ex.bn1 <- model2network("[X1][X2|X1][X3|X2]")
graphviz.plot(ex.bn1,layout = 'neato')

In [ ]:
# This function computes the humongous formula for P(B,D)
# This function is not general, I'm assuming that the variable Xj depends only on Xj-1.
# Of course it works for this particular case
ex.probmodel1 <- function(D)
    {
        nvar <- length(names(D))
        
        # For n = 1
        col1 <- dplyr::pull(D, 1)
        nunique1 <- length(unique(col1))
        prod <- factorial(nunique1-1)
        den <- nunique - 1
        
        for(i in 1:nunique1)
            {
                prod <- prod*factorial(length(col1[col1 == unique(col1)[i]]))
                den <- den + length(col1[col1 == unique(col1)[i]])
            }
        prod <- prod/factorial(den)
    
        # For n >= 1
        for(i in 2:nvar)
            {
                col <- dplyr::pull(D, i)
                prevcol <- dplyr::pull(D, i-1)
                
                r <- length(unique(col))
                q <- length(unique(prevcol))
                for(j in 1:q)
                    {
                        nij <- 0 
                        for(k in 1:r)
                            {
                                nijk <- sum(prevcol == unique(prevcol)[j] & col == unique(col)[k])
                                nij <- nij + nijk
                                prod <- prod*factorial(nijk)
                            }
                    
                        prod <- prod*factorial(r - 1)/factorial(nij + r - 1)
                    }
                
            }
    
        return(prod)
    }

ex.probmodel1(ex)

In [ ]:
ex.bn2 <- model2network("[X1][X2|X1][X3|X1]")
graphviz.plot(ex.bn2)

In [ ]:
# This function is very similar for the previous one but it's adapted for this other network structure
ex.probmodel2 <- function(D)
    {
        nvar <- length(names(D))
        
        # For n = 1
        col1 <- dplyr::pull(D, 1)
        nunique1 <- length(unique(col1))
        prod <- factorial(nunique1-1)
        den <- nunique1 - 1
        
        for(i in 1:nunique1)
            {
                prod <- prod*factorial(length(col1[col1 == unique(col1)[i]]))
                den <- den + length(col1[col1 == unique(col1)[i]])
            }
        prod <- prod/factorial(den)
    
        # For n >= 1
        for(i in 2:nvar)
            {
                col <- dplyr::pull(D, i)
                prevcol <- dplyr::pull(D, 1)
                
                r <- length(unique(col))
                q <- length(unique(prevcol))
                for(j in 1:q)
                    {
                        nij <- 0 
                        for(k in 1:r)
                            {
                                nijk <- sum(prevcol == unique(prevcol)[j] & col == unique(col)[k])
                                nij <- nij + nijk
                                prod <- prod*factorial(nijk)
                            }
                    
                        prod <- prod*factorial(r - 1)/factorial(nij + r - 1)
                    }
                
            }
    
        return(prod)
    }

ex.probmodel2(ex)

In [ ]:
ex.probmodel.general <- function(BN,D)
    {
        nvar <- length(nodes(BN))
        prod <- 1
        ex.prob.noparents <- function(namecol,prod)
            {
                col <- dplyr::pull(D, namecol)
                
                nunique <- length(unique(col))
                prod <- prod*factorial(nunique-1)
                den <- nunique - 1
            
                for(i in 1:nunique)
                    {
                        prod <- prod*factorial(length(col[col == unique(col)[i]]))
                        den <- den + length(col[col == unique(col)[i]])
                    }
            
                prod <- prod/factorial(den)
            
                return(prod)
            }
    
        ex.prob.parents <- function(namecol,prod)
            {
                col <- dplyr::pull(D, namecol)
                parents <- parents(BN, namecol)
                n.parents <- length(parents)
                col.parents <- D[parents]
                r <- length(unique(col))
    
                q <- 1
                combined <- list()
                for(j in 1:length(parents))
                    {
                        q <- q*length(unique(col.parents[,j]))
                        combined[[j]] <- unique(col.parents[,j])    
                    }
                combinations <- do.call(expand.grid, combined)
                # for j in 1:qi
                for(j in 1:q)
                    {
                        w  <- combinations[j,]
                        # Compute Nijk!
                        nij <- 0
                        for(k in 1:r)
                            {
                                wij <- c(w,unique(col)[k])
                                nijk <- sum(apply(apply(cbind(col.parents, col),1,is.eq,row2=wij),2,all))
                                nij <- nij + nijk
                                prod <- prod*factorial(nijk)
                            }
                        prod <- prod*factorial(r - 1)/factorial(nij + r - 1)
                    }
            
                return(prod)
            
            }
        
        for(i in 1:nvar)
            {
                if(length(parents(BN, nodes(BN)[i])) == 0)
                    {prod <- ex.prob.noparents(nodes(BN)[i],prod)}
                else
                    {prod <- ex.prob.parents(nodes(BN)[i],prod)}
            }
    
        return(prod)
    }

In [ ]:
ex.probmodel.general(ex.bn1,ex)

In [ ]:
ex.probmodel.general(ex.bn2,ex)

The general function seem to work well since it reproduces the same values of the paper aswell!